In [298]:
import pandas as pd
import plotly.express as px

time_series_confirmed = pd.read_csv('time_series_covid_19_confirmed.csv')

In [299]:
dates = []
from datetime import date, timedelta

start_date = date(2020, 2, 14)
end_date = date(2020, 3, 10)
delta = timedelta(days=1)
while start_date <= end_date:
    dates.append(start_date.strftime("%-m/%-d/20"))
    start_date += delta

In [300]:
import plotly.graph_objects as go
fig = go.Figure()

time_series_confirmed = time_series_confirmed[time_series_confirmed['Country/Region']!="Mainland China"]
time_series_confirmed = time_series_confirmed[time_series_confirmed['Country/Region']!="Others"]

affected_countries = time_series_confirmed['Country/Region'].unique()

for eachCountry in affected_countries:
    df_country = time_series_confirmed[time_series_confirmed['Country/Region'] == eachCountry]
    x_c, y_c = [], []
    for eachDate in dates:
        x_c.append(eachDate)
        y_c.append(df_country[eachDate].sum())
    fig.add_trace(go.Scatter(x=x_c, y=y_c, mode='lines', name=eachCountry))

In [301]:
fig.show()

In [302]:
fig1 = go.Figure()

time_series_confirmed = time_series_confirmed[time_series_confirmed[eachDate]>500]

for eachCountry in affected_countries:
    df_country = time_series_confirmed[time_series_confirmed['Country/Region'] == eachCountry]
    x_c, y_c = [], []
    for eachDate in dates:
        cases_count = df_country[eachDate].sum()
        if cases_count >= 100:
            x_c.append(eachDate)
            y_c.append(df_country[eachDate].sum())
    fig1.add_trace(go.Scatter(x=x_c, y=y_c, mode='lines', name=eachCountry))

In [303]:
fig1.show()

In [376]:
open_line_list = pd.read_csv('COVID19_open_line_list.csv')

In [377]:
unique_lat = open_line_list['latitude'].unique()
unique_lon = open_line_list['longitude'].unique()

In [378]:
ll_cnt = {}
ha_ll = {}

max_val = 0
min_val = 10

for index, eachRow in open_line_list.iterrows():
    key = (eachRow["latitude"], eachRow["longitude"]).__hash__()
    if ll_cnt.get(key) is None:
        ll_cnt[key]=0
    ll_cnt[key]=ll_cnt.get(key)+1
    if ll_cnt[key] > max_val:
        max_val = ll_cnt[key]
    if ll_cnt[key] < min_val:
        min_val = ll_cnt[key]
    ha_ll[key] = (eachRow["latitude"], eachRow["longitude"])

global_map = []
for key in ha_ll:
    old_val_min = 0
    old_val_max = max_val - min_val
    rangex = old_val_max - old_val_min
    percent = ((ll_cnt[key]-old_val_min)/old_val_max)*100
    
    new_red_val = 94+(100-percent)
    new_green_val = 0+(100-(percent*158/100))
    
    rangex = max_val - min_val
    color = "rgb(%d,%d,255)" % (new_red_val, new_green_val)
    global_map.append([ha_ll[key][0], ha_ll[key][1], ll_cnt[key], color])

global_map_df = pd.DataFrame(global_map, columns = ['lat', 'long', 'cases', 'color'])

print(global_map_df.head())

         lat        long  cases            color
0  31.646960  117.716600      2  rgb(193,99,255)
1  31.778630  117.331900      3  rgb(193,99,255)
2  31.828313  117.224844      2  rgb(193,99,255)
3  32.001230  117.568100      8  rgb(193,98,255)
4  31.759410  116.315100     53  rgb(188,91,255)


In [379]:
fig2 = go.Figure(data=go.Scattergeo(
        lon = global_map_df['long'],
        lat = global_map_df['lat'],
        text = global_map_df['cases'],
        mode = 'markers',
        marker = dict(
            size = 0,
            opacity = 0.1,
            autocolorscale = True,
            symbol = 'circle-dot',
            line = dict(
                width=0
            ),
            colorscale = 'geyser',
            cmin = global_map_df['cases'].min(),
            color = global_map_df['cases'],
            cmax = global_map_df['cases'].max(),
        )))

fig2.update_layout(
        title = 'CoVid-19 cases<br>(Hover for number of cases)',
        geo = dict(
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5
        ),
    )
fig2.show()

In [380]:
print(open_line_list['sex'].unique())
print(len(open_line_list))

['male' 'female' nan '4000' 'Female' 'Male']
14126


In [381]:
labels = ['male','female']

male = open_line_list[open_line_list['sex']=='male'] + open_line_list[open_line_list['sex']=='Male']
female = open_line_list[open_line_list['sex']=='female'] + open_line_list[open_line_list['sex']=='Female']

values = [len(male), len(female)]

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.show()